**Cargar librerias necesarias**

In [40]:
import numpy as np
import pandas as pd
import tensorflow as tf

**Cargar datos**

In [41]:
df = pd.read_csv('listings_limpio.csv')
df.head()

,id,host_response_rate,host_acceptance_rate,neighbourhood_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,...,Safe,Refrigerator,Smoke_alarm_home_safety,Essentials,Services,property_Entire_Place,property_Hotel_Room,property_Other,property_Private_Room,property_Shared_Room
0,30320,100,100,Sol,40.41476,-3.70418,2,1.0,1.0,2.0,...,0,0,0,0,0,True,False,False,False,False
1,40916,100,100,Universidad,40.42247,-3.70577,2,1.0,1.0,3.0,...,0,0,0,0,0,True,False,False,False,False
2,62423,100,99,Justicia,40.41884,-3.69655,4,1.5,1.0,2.0,...,0,1,1,1,0,False,False,False,True,False
3,70073,100,100,Universidad,40.42252,-3.70250,2,1.0,1.0,1.0,...,0,1,0,1,1,True,False,False,False,False
4,70310,100,100,Universidad,40.42252,-3.70250,2,1.0,1.0,2.0,...,0,1,0,1,1,True,False,False,False,False


Eliminar "neighbourhood_cleansed" ya que es redundante contar con esto al tener la longitu y latitud

In [42]:
df["neighbourhood_cleansed"]
df.drop(columns=["neighbourhood_cleansed"], inplace=True)

Codificar booleanas como 1 y 0 

In [43]:
bool_cols = df.select_dtypes(include="bool").columns
df[bool_cols] = df[bool_cols].astype(int)

df.head()

,id,host_response_rate,host_acceptance_rate,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,...,Safe,Refrigerator,Smoke_alarm_home_safety,Essentials,Services,property_Entire_Place,property_Hotel_Room,property_Other,property_Private_Room,property_Shared_Room
0,30320,100,100,40.41476,-3.70418,2,1.0,1.0,2.0,157.0,...,0,0,0,0,0,1,0,0,0,0
1,40916,100,100,40.42247,-3.70577,2,1.0,1.0,3.0,143.0,...,0,0,0,0,0,1,0,0,0,0
2,62423,100,99,40.41884,-3.69655,4,1.5,1.0,2.0,65.0,...,0,1,1,1,0,0,0,0,1,0
3,70073,100,100,40.42252,-3.70250,2,1.0,1.0,1.0,116.0,...,0,1,0,1,1,1,0,0,0,0
4,70310,100,100,40.42252,-3.70250,2,1.0,1.0,2.0,79.0,...,0,1,0,1,1,1,0,0,0,0


**Separar en train y test**

In [44]:
train = df.sample(frac=0.8, random_state=100)
train.head()

,id,host_response_rate,host_acceptance_rate,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,...,Safe,Refrigerator,Smoke_alarm_home_safety,Essentials,Services,property_Entire_Place,property_Hotel_Room,property_Other,property_Private_Room,property_Shared_Room
2415,32892473,100,92,40.406490,-3.745560,4,1.0,1.0,1.0,117.0,...,0,1,1,1,1,1,0,0,0,0
11422,1183135383050248925,50,40,40.421990,-3.702730,3,1.0,1.0,1.0,158.0,...,0,0,0,0,0,1,0,0,0,0
13907,1334659658421449266,33,17,40.473280,-3.704860,1,1.0,1.0,1.0,34.0,...,0,0,0,0,0,0,0,0,1,0
2120,29759801,100,100,40.333614,-3.702798,2,1.0,1.0,2.0,93.0,...,0,1,1,1,1,1,0,0,0,0
5370,664313376708004518,99,99,40.447750,-3.708190,4,1.0,1.0,2.0,118.0,...,0,1,0,1,1,1,0,0,0,0


In [45]:
test = df.drop(train.index)
test.head()

,id,host_response_rate,host_acceptance_rate,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,...,Safe,Refrigerator,Smoke_alarm_home_safety,Essentials,Services,property_Entire_Place,property_Hotel_Room,property_Other,property_Private_Room,property_Shared_Room
26,339171,100,100,40.41285,-3.69553,4,1.0,1.0,2.0,157.0,...,0,1,1,1,1,1,0,0,0,0
27,342095,100,81,40.41015,-3.69797,2,1.0,1.0,2.0,95.0,...,0,1,1,1,1,1,0,0,0,0
40,469231,86,95,40.41941,-3.70823,2,1.0,1.0,1.0,139.0,...,0,1,0,1,1,1,0,0,0,0
45,502918,100,100,40.41155,-3.69492,1,1.0,2.0,1.0,50.0,...,0,1,1,1,1,0,0,0,1,0
48,510973,83,8,40.41412,-3.70057,1,2.0,1.0,3.0,101.0,...,0,1,0,1,0,0,0,0,1,0


In [46]:
print(train.shape)
print(test.shape)

(14131, 39)
(3533, 39)


**Separar X y Y**

In [47]:
y_train = train['price']
y_test = test['price']
x_train = train.drop(columns=['price'])
x_test = test.drop(columns=['price'])

**Construir el modelo usando tensorfloww**

capa de normalización

In [48]:
tf.keras.layers.Normalization

keras.src.layers.preprocessing.normalization.Normalization

In [49]:
norm = tf.keras.layers.Normalization()
norm.adapt(np.array(x_train))
norm.mean.numpy()
norm.variance.numpy()
np.sqrt(norm.variance.numpy())
norm(x_train).numpy()

array([[-1.5451945 ,  0.4210826 ,  0.23352788, ..., -0.05327937,
        -0.5463031 , -0.07870716],
       [ 0.63977593, -1.882206  , -1.7946874 , ..., -0.05327937,
        -0.5463031 , -0.07870716],
       [ 0.91960526, -2.6653242 , -2.6917827 , ..., -0.05327937,
         1.8304855 , -0.07870716],
       ...,
       [-0.4107798 ,  0.4210826 , -1.5216584 , ..., -0.05327937,
        -0.5463031 , -0.07870716],
       [ 1.127398  ,  0.4210826 ,  0.07751133, ..., -0.05327937,
         1.8304855 , -0.07870716],
       [ 0.16776411,  0.4210826 , -0.4295425 , ..., -0.05327937,
         1.8304855 , -0.07870716]], dtype=float32)